In [ ]:
import os
import json
from pathlib import Path
from rich.pretty import pprint
from openai import OpenAI
from docai import utils

In [ ]:
GDX_ARTIFACTS = Path("/Users/kwesi/Desktop/ai/gdxf/src/artifacts")
loe_document = open(GDX_ARTIFACTS / "texts/data/ps_ross_whitehead.txt").read()
loe_schema = json.load(open(GDX_ARTIFACTS / "schemas/data/loe_detailed.json"))
ps_document = open(GDX_ARTIFACTS / "texts/data/ps_ross_whitehead.txt").read()
ps_schema = json.load(open(GDX_ARTIFACTS / "schemas/data/ps_detailed.json"))
crm_document = open(GDX_ARTIFACTS / "texts/data/crm_wes_harden.txt").read()
crm_schema = json.load(open(GDX_ARTIFACTS / "schemas/data/crm_debrief.json"))
tools = [
    
    {
        "type": "function",
        "function": {
            "name": "get_letter_of_employment_data",
            "description": "Extracts data from a letter of employment",
            "parameters": loe_schema
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_pay_slip_data",
            "description": "Extracts data from a pay slip",
            "parameters": ps_schema
        }
    },
    # {
    #     "type": "function",
    #     "function": {
    #         "name": "get_crm_data",
    #         "description": "Extracts data from a Customer Relationship Management software notes",
    #         "parameters": crm_schema
    #     }
    # }
]

In [ ]:
env = {
    "SCHEMA_TABLE_PARAMETER_NAME": "/dev/table/schema_table_name",
    "OPENAI_API_KEY_PARAMETER_NAME": "/dev/secret/openai/api_key"
}
os.environ.update(env)
resource = utils.Resources()
secrets = utils.Secrets()
schema_table = resource.get_table("SCHEMA_TABLE_PARAMETER_NAME")
client = OpenAI(api_key=secrets("OPENAI_API_KEY_PARAMETER_NAME"))

In [ ]:
user_content = [
    {"type": "text", "text": f"I need help extracting data from an employment letter. \n\n Employment Letter: \n\n {loe_document}"},
]
messages = [
    {"role": "user", "content": user_content}
]
response_1 = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    tools=tools,
)
pprint(json.loads(response_1.choices[0].message.tool_calls[0].function.arguments))

In [ ]:
user_content = [
    {"type": "text", "text": f"I need help extracting data from a pay stub. \n\n Pay Stub: \n\n {ps_document}"},
]
messages = [
    {"role": "user", "content": user_content}
]
response_2 = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    tools=tools,
)
pprint(json.loads(response_2.choices[0].message.tool_calls[0].function.arguments))

In [ ]:
# user_content = [
#     {"type": "text", "text": f"I need help extracting data from notes for a customer relationship management software. \n\n CRM Notes: \n\n {crm_document}"},
# ]
# messages = [
#     {"role": "user", "content": user_content}
# ]
# response_3 = client.chat.completions.create(
#     model="gpt-4",
#     messages=messages,
#     tools=tools,
# )
# pprint(response_3.choices[0].message.tool_calls[0])